In [1]:
import numpy as np
import sympy

# Question 3
class manipulator is a manipulator class which has the option to change the configurations according to the users requirement. More details of the same are explained in the code comments

In [2]:
class manipulator():
    def __init__(self,dh_params, config=['R','R']):
        # configuration of the manipulator. User has 2 choices "R"->revolute. "P"->prismatic.
        # Default configuration is a 2R manipulator with all the angles at 0 degrees and lengths being 1 unit.
        self.config = config 
        # User must input the dh parameters in matrix form i.e. "R"->revolute
        # [[a1 , alpha1 , d1, theta1]
        #  [a2 , alpha2 , d2, theta2]
        #  .
        #  .
        #  .
        #  [an , alphan , dn, thetan]]
        # n being the nth link of the manipulator.
        self.dh=dh_params
    
    def calc_tranfMatrix(self, dh_params,i):
        # Calculating Trnasformation matrix
        a, alpha,d,theta = dh_params
        A = np.array([[np.cos(theta), -np.sin(theta)*np.cos(alpha),  np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
                      [np.sin(theta),  np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
                      [            0,                np.sin(alpha),                np.cos(alpha),               d],
                      [            0,                            0,                            0,               1]])
        return A 

    def forward_kinematics(self,):
        tr=self.calc_tranfMatrix(self.dh[0],0)
        Trs = [tr]
        # Calculating the individual transformation matrices. And appending to the T matrix in the following form.
        # A1
        # A1A2
        # A1A2A3 ... 
        for i in range(len(self.dh)-1):
            tr = np.matmul(tr,self.calc_tranfMatrix(self.dh[i+1],i+1))
            Trs.append(tr)

        # Calculating the jacobian matrix
        h = []
        for i in range(len(self.config)):
            temp2 = np.array([0,0,0])
            if self.config[i]=='R':
                temp2 = np.array([0,0,1])
            if  i ==0:
                temp = np.array(Trs[-1])

            else:
                temp = np.array(Trs[-1]) - np.array(Trs[i-1])
            
            h.append(np.cross(temp2,temp[:3,3:].transpose()).transpose())
        
        # Velocity jacobian
        J_v = h[0]
        for i in range(len(self.config)-1):
            J_v = np.hstack((J_v,h[i+1]))
        
        
        # Angular velocity jacobian
        J_omega = np.array([[0],[0],[1]])
        if self.config[0]=='P':
            J_omega = np.array([[0],[0],[0]])

        for i in range(len(self.config)-1):
            temp = np.array([[0],[0],[1]])

            if self.config[i+1]=='P':
                temp = np.array([[0],[0],[0]])
            J_omega = np.hstack((J_omega,temp))
        
        # Overall Transformation matrix T06.
        transformation_matrix = np.array(Trs[-1])
        # Manipulator jacobian
        J = np.vstack((J_v,J_omega))

        return {'transformation_matrix':transformation_matrix,'jacobian':J}
        
        

In [3]:
'''
Example of 3R revokute manipulator
'''
manipulator_config = ['R','R','R']
dh_params = [[1,0,0,np.pi/3],
             [1,0,0,0],
             [1,0,0,0]]

mani = manipulator(dh_params,manipulator_config)
a = mani.forward_kinematics()

print(np.round(a['transformation_matrix'].astype('float'),3))
print(np.round(a['jacobian'].astype('float'),3))

# Velocity of end effector
#  Use the following code by giving an input qdots.
'''
qdots = [0]*len(manipulator_config)
vels = np.matmul((a['jacobian'],qdots.T))
'''


[[ 0.5   -0.866  0.     1.5  ]
 [ 0.866  0.5    0.     2.598]
 [ 0.     0.     1.     0.   ]
 [ 0.     0.     0.     1.   ]]
[[-2.598 -1.732 -0.866]
 [ 1.5    1.     0.5  ]
 [ 0.     0.     0.   ]
 [ 0.     0.     0.   ]
 [ 0.     0.     0.   ]
 [ 1.     1.     1.   ]]


"\nqdots = [0]*len(manipulator_config)\nvels = np.matmul((a['jacobian'],qdots.T))\n"

# Question 4
Verifying the above code for Stanford and Scara manipulators. The dh parameters are taken from Spong and Vidyasagr book.

In [4]:
# Stanford Configuration
manipulator_config = ['R','R','P']
# a,alpha,d,theta
dh_params = [[0,-np.pi/2,0,0],
             [0,np.pi/2,1,0],
             [0,0,1,0]]

mani = manipulator(dh_params,manipulator_config)
a = mani.forward_kinematics()

print(np.round(a['transformation_matrix'].astype('float'),3))
print(np.round(a['jacobian'].astype('float'),3))

[[1. 0. 0. 0.]
 [0. 1. 0. 1.]
 [0. 0. 1. 1.]
 [0. 0. 0. 1.]]
[[-1. -1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 1.  1.  0.]]


In [5]:
#  Scara Manipulator
manipulator_config = ['R','R','P']
dh_params = [[1,0,0,0],
             [1,np.pi,0,0],
             [0,0,1,0]]

mani = manipulator(dh_params,manipulator_config)
a = mani.forward_kinematics()

print(np.round(a['transformation_matrix'].astype('float'),3))
print(np.round(a['jacobian'].astype('float'),3))

[[ 1.  0.  0.  2.]
 [ 0. -1. -0. -0.]
 [ 0.  0. -1. -1.]
 [ 0.  0.  0.  1.]]
[[ 0.  0.  0.]
 [ 2.  1.  0.]
 [-0. -0. -0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 1.  1.  0.]]


# Question 5


In [58]:
manipulator_config = ['P','P','P']
dh_params = [[0,-np.pi/2,1,0],
             [0,np.pi/2,1,np.pi/2],
             [0,0,1,-np.pi/2]]

mani = manipulator(dh_params,manipulator_config)
a = mani.forward_kinematics()

print(np.round(a['transformation_matrix'].astype('float'),3))
print(np.round(a['jacobian'].astype('float'),3))

[[ 0.  0.  1.  1.]
 [-1.  0.  0.  1.]
 [-0. -1.  0.  1.]
 [ 0.  0.  0.  1.]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


# Question 6

In [6]:
manipulator_config = ['R','R','R','R','R','R']
dh_params = [[0,np.pi/2,0,0],
             [1,0,0,0],
             [1,0,0,0],
             [0,-np.pi/2,0,0],
             [0,0,0,0],
             [0,0,1,0]]

mani = manipulator(dh_params,manipulator_config)
a = mani.forward_kinematics()
print("######################")
print(np.round(a['transformation_matrix'].astype('float'),3))
print(np.round(a['jacobian'].astype('float'),3))

######################
[[1. 0. 0. 2.]
 [0. 1. 0. 0.]
 [0. 0. 1. 1.]
 [0. 0. 0. 1.]]
[[0. 0. 0. 0. 0. 0.]
 [2. 2. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1.]]


# Question 11


In [ ]:
# qs is list of elements of the type sympy.physics.dynamicsymbols('q1 q2 ....') where q1,q2 are generalised coordinates
# qdpt_matrix is a matrix of derivatves of the generalised coordinates
def calcEqns(D,V,q_matrix,qdot_matrix,qs):
    K = 1/2*qdot_matrix.T*D*qdot_matrix
    L = K - V
    LM = sympy.physics.mechanics.LagrangesMethod(L,qs)
    # LM Matrix gives the lagrangian equation in matrix form

